In [67]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [68]:
temp_df=pd.read_csv('IMDB review Dataset.csv')

In [69]:
df=temp_df.iloc[:10000]

In [70]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [71]:
df['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [72]:
df['sentiment'].value_counts()

positive    5028
negative    4972
Name: sentiment, dtype: int64

In [73]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

In [74]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [75]:
df.duplicated().sum() #There are 17 duplicate reviews

17

In [76]:
df.drop_duplicates(inplace=True)

In [77]:
df.duplicated().sum()

0

In [78]:
# Basic Preprocessing
# Remove Tags
# LowerCase
# Remove stopwords

In [79]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<.*?\>'),'',raw_text)
    return cleaned_text

In [80]:
df['review']=df['review'].apply(remove_tags)

In [81]:
df['review']=df['review'].apply(lambda x:x.lower())

In [82]:
import nltk

In [83]:
 #nltk.download('stopwords')

In [84]:
#stopword removal

from nltk.corpus import stopwords

sw_list=stopwords.words('english')

df['review']=df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:' '.join(x))

In [85]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,"fun, entertaining movie wwii german spy (julie...",positive
9996,"give break. anyone say ""good hockey movie""? kn...",negative
9997,movie bad movie. watching endless series bad h...,negative
9998,"movie probably made entertain middle school, e...",negative


In [86]:
df['review'][1]

'wonderful little production. filming technique unassuming- old-time-bbc fashion gives comforting, sometimes discomforting, sense realism entire piece. actors extremely well chosen- michael sheen "has got polari" voices pat too! truly see seamless editing guided references williams\' diary entries, well worth watching terrificly written performed piece. masterful production one great master\'s comedy life. realism really comes home little things: fantasy guard which, rather use traditional \'dream\' techniques remains solid disappears. plays knowledge senses, particularly scenes concerning orton halliwell sets (particularly flat halliwell\'s murals decorating every surface) terribly well done.'

In [87]:
X=df.iloc[:,0:1]
y=df['sentiment']

In [88]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [89]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [90]:
from sklearn.preprocessing import LabelEncoder

encoder=LabelEncoder()

y=encoder.fit_transform(y)

In [91]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [92]:
X_train.shape

(7986, 1)

In [93]:
X_test.shape

(1997, 1)

In [94]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [95]:
X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()

In [96]:
X_train_bow.shape

(7986, 48282)

In [100]:
X_test_bow.shape

(1997, 48282)

In [99]:
from sklearn.naive_bayes import GaussianNB
gnb=GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [101]:
y_pred=gnb.predict(X_test_bow)

In [102]:
y_pred

array([1, 1, 0, ..., 1, 0, 0])

In [103]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6324486730095142

In [104]:
confusion_matrix(y_test,y_pred)

array([[717, 235],
       [499, 546]], dtype=int64)

In [105]:
#Since our accuracy is not that high, we are going to apply another algorithm.

In [106]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8537806710065098

In [107]:
cv=CountVectorizer(max_features=3000)

X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()

rf=RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

#accuracy slightly reduced since we have reduced the vocabulary

0.8447671507260891

In [108]:
cv=CountVectorizer(ngram_range=(1,2))

X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()

rf=RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

#lack of space

MemoryError: Unable to allocate 44.5 GiB for an array with shape (7986, 748659) and data type int64

In [109]:
cv=CountVectorizer(ngram_range=(1,2),max_features=3000)

X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()

rf=RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8387581372058087

# Using TF-IDF

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [113]:
tfidf=TfidfVectorizer()

X_train_tfidf=tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf=tfidf.transform(X_test['review'])

rf=RandomForestClassifier()

tfidf.fit(X_train_tfidf,y_train)
y_pred=tfidf.predict(X_test_tfidf)
accuracy_score(y_test,y_pred)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

# Word to vec

In [114]:
import numpy as np
import pandas as pd

In [116]:
temp_df=pd.read_csv('IMDB review Dataset.csv')

In [117]:
df=temp_df[:10000]

In [118]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [119]:
df.drop_duplicates(inplace=True)

In [120]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<.*?\>'),'',raw_text)
    return cleaned_text

In [121]:
df['review']=df['review'].apply(remove_tags)

In [122]:
df['review']=df['review'].apply(lambda x:x.lower())

In [123]:
#stopword removal

from nltk.corpus import stopwords

sw_list=stopwords.words('english')

df['review']=df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:' '.join(x))

In [124]:
import gensim

In [125]:
from nltk import sent_tokenize # sent_tokenize ---> sentence tokenization
from gensim.utils import simple_preprocess

In [126]:
story=[]
for doc in df['review']:
    raw_sent=sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [128]:
model=gensim.models.Word2Vec(window=10,min_count=2) #min 2 words must we present in the sentence #initializing object
#100 dimension vectors will be formed by deafult

In [131]:
model.build_vocab(story)

In [132]:
model.train(story,total_examples=model.corpus_count, epochs=model.epochs)

(5875770, 6212140)

In [134]:
len(model.wv.index_to_key)

31845

In [135]:
model.wv.index_to_key

#words in vocabulary

['movie',
 'film',
 'one',
 'like',
 'good',
 'it',
 'the',
 'would',
 'time',
 'even',
 'story',
 'see',
 'really',
 'well',
 'much',
 'get',
 'bad',
 'great',
 'people',
 'first',
 'also',
 'made',
 'make',
 'way',
 'movies',
 'could',
 'think',
 'characters',
 'watch',
 'character',
 'films',
 'that',
 'never',
 'little',
 'show',
 'seen',
 'many',
 'two',
 'love',
 'acting',
 'plot',
 'best',
 'know',
 'life',
 'this',
 'ever',
 'better',
 'man',
 'there',
 'still',
 'say',
 'scene',
 'end',
 'and',
 'scenes',
 'something',
 'go',
 'real',
 'back',
 'watching',
 'director',
 'actors',
 'years',
 'thing',
 'though',
 've',
 'work',
 'look',
 'funny',
 'actually',
 'old',
 'nothing',
 'going',
 'makes',
 'new',
 'lot',
 'another',
 'all',
 'every',
 'find',
 'pretty',
 'things',
 'part',
 'can',
 'he',
 'us',
 'world',
 'horror',
 'around',
 'want',
 'big',
 'quite',
 'cast',
 'long',
 'young',
 'enough',
 'in',
 'take',
 'seems',
 'got',
 'must',
 'however',
 'may',
 'thought',
 'fa

In [140]:
#code for converting sentences to vector
def document_vector(doc):
    doc=[word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc],axis=0)

In [143]:
document_vector(df['review'].values[0]) #converting the first review to vector

array([-0.15480669,  0.5123571 ,  0.7093771 ,  0.01027348, -0.22482702,
       -0.10647601, -0.35621446,  0.5422294 ,  0.08043259,  0.33000824,
       -0.16184959,  0.02080165, -0.32924643, -0.15783866,  0.17598218,
       -0.289469  , -0.09516775, -0.54589766,  0.20991078, -0.47521502,
        0.281658  ,  0.10068011,  0.36618403,  0.07331216, -0.23769316,
        0.24502644,  0.11451492, -0.13874428, -0.670655  ,  0.47252828,
       -0.21287984,  0.14188768,  0.264387  ,  0.20148517,  0.2983174 ,
       -0.39522105,  0.08999523, -0.4240164 , -0.3897448 , -0.32696787,
       -0.20440519, -0.07944923, -0.21955466,  0.01643202, -0.05720714,
       -0.14697482, -0.18816398,  0.30110356, -0.10487099,  0.5322978 ,
       -0.05295952,  0.15473332, -0.11993817,  0.13147861, -0.34566444,
        0.16973987,  0.36161605,  0.4039922 , -0.18764554,  0.10735478,
       -0.3668434 ,  0.17806622, -0.5023229 ,  0.07616809, -0.01650833,
       -0.18446973,  0.05260963, -0.2723754 , -0.18259984, -0.13

In [ ]:
from tqdm import tqdm

In [147]:
X=[]
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [15:08<00:00, 10.99it/s]


In [148]:
X=np.array(X)

In [149]:
X.shape #there are 9983 reviews, hence 9983 vectors, each of 100 dimensions

(9983, 100)

In [151]:
from sklearn.preprocessing import LabelEncoder

In [152]:
y=encoder.fit_transform(df['sentiment'])

In [153]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [154]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [155]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [156]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7696544817225839